In [1]:
import pandas as pd
import numpy as np

# Load data
user_behaviour = pd.read_csv("user_behaviour.csv")
revenuecat = pd.read_csv("revenuecat.csv")

user_behaviour.head(), user_behaviour.shape, revenuecat.shape


(   event_date                    user_pseudo_id  \
 0  2025-07-22  018CCC8F1ED3479E86B2C8BEAA649FCE   
 1  2025-07-22  018CCC8F1ED3479E86B2C8BEAA649FCE   
 2  2025-07-22  018CCC8F1ED3479E86B2C8BEAA649FCE   
 3  2025-07-22  018CCC8F1ED3479E86B2C8BEAA649FCE   
 4  2025-07-22  018CCC8F1ED3479E86B2C8BEAA649FCE   
 
                      revenue_cat_id        country              city version  \
 0  018CCC8F1ED3479E86B2C8BEAA649FCE  United States  Huntington Beach   1.6.2   
 1  018CCC8F1ED3479E86B2C8BEAA649FCE  United States  Huntington Beach   1.6.2   
 2  018CCC8F1ED3479E86B2C8BEAA649FCE  United States  Huntington Beach   1.6.2   
 3  018CCC8F1ED3479E86B2C8BEAA649FCE  United States  Huntington Beach   1.6.2   
 4  018CCC8F1ED3479E86B2C8BEAA649FCE  United States  Huntington Beach   1.6.2   
 
   device_name test_variant             event_name feature paywall_type  \
 0      iPhone        Plans             first_open     NaN          NaN   
 1      iPhone        Plans          session_sta

In [2]:
# Quick check of variants
print(user_behaviour['test_variant'].value_counts())

# One row per user with variant
variant_per_user = user_behaviour[['revenue_cat_id', 'test_variant']].drop_duplicates()
variant_per_user.head(), variant_per_user.shape


test_variant
Standard    62349
Plans       54036
Name: count, dtype: int64


(                      revenue_cat_id test_variant
 0   018CCC8F1ED3479E86B2C8BEAA649FCE        Plans
 22  083A134A7E954DB9AA4A9CE4F6A95291     Standard
 64  0AC4A12289FE41E6B80700114A091446     Standard
 81  0DCC141028B943818A7C21E01BCC4867     Standard
 94  12D85277102A44ECB5614913CAF20A08     Standard,
 (3183, 2))

In [3]:

# Total users per variant (distinct revenue_cat_id)
total_users = (
    user_behaviour
    .groupby('test_variant')['revenue_cat_id']
    .nunique()
    .rename('total_users')
)

# Paying users per variant (at least one payment_success)
paying_users = (
    user_behaviour[user_behaviour['event_name'] == 'payment_success']
    .groupby('test_variant')['revenue_cat_id']
    .nunique()
    .rename('paying_users')
)

# Conversion table
conversion = (
    pd.concat([total_users, paying_users], axis=1)
    .assign(conversion_rate=lambda df: df['paying_users'] / df['total_users'])
    .reset_index()
)

conversion


,test_variant,total_users,paying_users,conversion_rate
0,Plans,1637,260,0.158827
1,Standard,1544,405,0.262306


In [4]:
purchases_by_paywall = (
    user_behaviour[user_behaviour['event_name'] == 'payment_success']
    .groupby(['test_variant', 'paywall_type'])['revenue_cat_id']
    .nunique()
    .reset_index(name='purchasing_users')
    .sort_values(['test_variant', 'purchasing_users'], ascending=[True, False])
)

purchases_by_paywall


,test_variant,paywall_type,purchasing_users
3,Plans,onboarding,231
0,Plans,feature,42
2,Plans,laggard,14
1,Plans,intended,4
7,Standard,onboarding,373
4,Standard,feature,48
6,Standard,laggard,24
5,Standard,intended,8
8,Standard,subscription_management,1


In [5]:
# Join test_variant into revenuecat
rev_enriched = revenuecat.merge(
    variant_per_user,
    on='revenue_cat_id',
    how='left'
)

# Net revenue = price * (1 - tax - commission)
rev_enriched['net_revenue'] = (
    rev_enriched['price_in_usd'] *
    (1 - rev_enriched['tax_percentage'] - rev_enriched['commission_percentage'])
)

# Keep only experiment subscriptions (with variant) and not refunded
rev_exp = rev_enriched[
    rev_enriched['test_variant'].notna() &
    rev_enriched['refunded_at'].isna()
].copy()

rev_enriched['test_variant'].value_counts(dropna=False), rev_exp.shape


(test_variant
 NaN         3172
 Standard     574
 Plans        339
 Name: count, dtype: int64,
 (890, 15))

In [6]:
# Net revenue per variant
net_rev_by_variant = rev_exp.groupby('test_variant')['net_revenue'].sum()

summary_rows = []
for variant in ['Standard', 'Plans']:
    total = int(total_users[variant])
    payers = int(paying_users[variant])
    netrev = float(net_rev_by_variant[variant])
    summary_rows.append({
        'variant': variant,
        'total_users': total,
        'paying_users': payers,
        'conversion_rate': payers / total,
        'total_net_revenue': netrev,
        'ARPU': netrev / total,
        'ARPPU': netrev / payers
    })

revenue_summary = pd.DataFrame(summary_rows)
revenue_summary


,variant,total_users,paying_users,conversion_rate,total_net_revenue,ARPU,ARPPU
0,Standard,1544,405,0.262306,3804.537827,2.464079,9.393921
1,Plans,1637,260,0.158827,3339.668074,2.040115,12.844877


In [7]:
# Normalised product base (remove _a/_b/_c at end)
rev_exp['product_base'] = rev_exp['product_identifier'].str.replace(r'_[abc]$', '', regex=True)

pkg_counts = (
    rev_exp
    .groupby(['test_variant', 'product_base'])['revenue_cat_id']
    .nunique()
    .reset_index(name='subscribers')
    .sort_values(['test_variant', 'subscribers'], ascending=[True, False])
)

pkg_counts


,test_variant,product_base,subscribers
3,Plans,gala_1wt_1w_gold,219
0,Plans,gala_1w_creator,20
4,Plans,gala_1y_creator,16
5,Plans,gala_1y_gold,16
1,Plans,gala_1w_immortal,1
2,Plans,gala_1w_master,1
6,Plans,gala_1y_immortal,1
10,Standard,gala_1wt_1w_gold,394
7,Standard,gala_1w_creator,18
11,Standard,gala_1y_creator,14


In [8]:
subs = rev_exp[['revenue_cat_id', 'test_variant', 'renewal_number']].copy()

max_renewal = (
    subs
    .groupby(['test_variant', 'revenue_cat_id'])['renewal_number']
    .max()
    .reset_index(name='max_renewal')
)

retention_summary = (
    max_renewal
    .groupby('test_variant')
    .agg(
        paying_users=('revenue_cat_id', 'nunique'),
        retention_1_plus=('max_renewal', lambda x: np.mean(x >= 1)),
        retention_2_plus=('max_renewal', lambda x: np.mean(x >= 2)),
        retention_3_plus=('max_renewal', lambda x: np.mean(x >= 3)),
    )
    .reset_index()
)

retention_summary


,test_variant,paying_users,retention_1_plus,retention_2_plus,retention_3_plus
0,Plans,251,0.203187,0.075697,0.023904
1,Standard,402,0.226368,0.116915,0.059701


In [9]:
conversion.to_csv("kpi_conversion.csv", index=False)
purchases_by_paywall.to_csv("kpi_purchases_by_paywall.csv", index=False)
revenue_summary.to_csv("kpi_revenue_summary.csv", index=False)
pkg_counts.to_csv("kpi_packages.csv", index=False)
retention_summary.to_csv("kpi_retention.csv", index=False)


In [ ]:
# 1) Filter to users who churned (rc_cancellation)
churn_events = user_behaviour[user_behaviour['event_name'] == 'rc_cancellation']

churn_users = churn_events['revenue_cat_id'].unique()

# 2) Keep only events for those users, up to and including first rc_cancellation
churn_data = user_behaviour[user_behaviour['revenue_cat_id'].isin(churn_users)].copy()

# Sort by user + time
churn_data = churn_data.sort_values(['revenue_cat_id', 'event_timestamp'])

# 3) For each user, truncate at first rc_cancellation
first_cancel_ts = (
    churn_data[churn_data['event_name'] == 'rc_cancellation']
    .groupby('revenue_cat_id')['event_timestamp']
    .min()
    .rename('cancel_ts')
)

churn_data = churn_data.merge(first_cancel_ts, on='revenue_cat_id', how='left')
churn_data = churn_data[churn_data['event_timestamp'] <= churn_data['cancel_ts']]

# 4) Build short flows: last 3 events before cancellation + rc_cancellation
def make_flow(events):
    # remove consecutive duplicates
    cleaned = []
    prev = None
    for e in events:
        if e != prev:
            cleaned.append(e)
        prev = e
    # take last 4 events (3 before + cancel if present)
    tail = cleaned[-4:]
    return " > ".join(tail)

flows = (
    churn_data
    .groupby('revenue_cat_id')['event_name']
    .apply(make_flow)
    .reset_index(name='flow')
)

# 5) Count users per flow and take top 10
top_flows = (
    flows
    .groupby('flow')['revenue_cat_id']
    .nunique()
    .reset_index(name='users')
    .sort_values('users', ascending=False)
    .head(10)
)

top_flows


,flow,users
262,tool_opened > show_paywalls > skip_paywalls > ...,27
65,image_generated > press_generate > image_gener...,26
195,show_paywalls > skip_paywalls > avatar_creatio...,19
101,payment_initiated > payment_cancelled > skip_p...,17
213,skip_paywalls > show_paywalls > skip_paywalls ...,16
111,payment_success > ftue_landmarkt > tool_opened...,14
208,skip_paywalls > avatar_creation > tool_opened ...,13
190,show_paywalls > payment_initiated > payment_ca...,12
199,show_paywalls > skip_paywalls > show_paywalls ...,12
12,avatar_creation > show_paywalls > skip_paywall...,11


In [11]:
top_flows.to_csv("kpi_churn_flows.csv", index=False)
